'''<br>
    @Author: v sanjay kumar<br>
    @Date: 2024-10-18 04:00:30<br>
    @Last Modified by: v sanjay kumar<br>
    @Last Modified time: 2024-010-018 04:00:30.<br>
    @Title :Check the reviews positive or negitive <br>

'''

In [68]:
import csv
import os
import google.generativeai as genai
from dotenv import load_dotenv

In [69]:
# Load environment variables
load_dotenv()

True

In [70]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Function to analyze sentiment and generate appropriate messages
def analyze_sentiment_and_generate_message(body):

    """
    Description:
    Analyze the sentiment of the review and generate a thank-you or apology message based on the sentiment.

    Parameters:
    body (str): The content of the review.

    Returns:
    tuple: A tuple containing:
        - sentiment_response (str): The sentiment of the review, either 'positive' or 'negative'.
        - response_message (str): A generated thank-you message for positive reviews or an apology message for negative reviews.
    """
    
    generation_config = {
        "temperature": 1,
        "max_output_tokens": 8192,
        "response_mime_type": "text/plain",
    }
    
    # Initialize the model
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config,
    )

    # Analyze sentiment
    chat_session = model.start_chat(history=[])
    prompt = f"Analyze the sentiment of the following review. Respond with only 'positive' or 'negative':\n\n{body}\n"
    sentiment_response = chat_session.send_message(prompt).text.strip()

    # Generate appropriate message
    if "positive" in sentiment_response.lower():
        message_prompt = f"Generate a short thank-you message for the following positive review:\n\n{body}\n"
    else:
        message_prompt = f"Generate an short apology message for the following negative review and say the we will improve accordingly:\n\n{body}\n"
    
    response_message = chat_session.send_message(message_prompt).text.strip()
    
    return sentiment_response, response_message



In [71]:
# Function to extract item and company
def extract_item_and_company(body):
    
    """
    Description:
    Extract the purchased item and the company name from the review.

    Parameters:
    body (str): The content of the review.

    Returns:
    tuple: A tuple containing:
        - item_response (str): The name of the purchased item, or 'unknown' if not identifiable.
        - company_response (str): The name of the company that made the item, or 'unknown' if not identifiable.
    """
    
    generation_config = {
        "temperature": 0.7,
        "max_output_tokens": 8192,
        "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config,
    )

    # Extract item
    chat_session = model.start_chat(history=[])
    item_prompt = f"Just Identify the purchased item in the following review: Respond with only the company name, such as 'Samsung phone':\n\n{body}\n"
    item_response = chat_session.send_message(item_prompt).text.strip() or "unknown"
    
    # Extract company
    company_prompt = f"Just identify the company name that made the item in the following review. Respond with only the company name, such as 'Samsung':\n\n{body}\n"
    company_response = chat_session.send_message(company_prompt).text.strip() or "unknown"

    return item_response, company_response




In [72]:
def main():
    # Read reviews from the text file
    reviews = []
    with open(r'C:\Users\sanju\bridgelab_local files\GENAi\review_input.txt', 'r') as file:
        review_content = file.read().strip().split('END')

    # Process each review
    for review in review_content:
        body = review.strip()

        # Extract item and company
        item, company = extract_item_and_company(body)
        
        # Analyze sentiment and generate message
        sentiment, response_message = analyze_sentiment_and_generate_message(body)
        
        # Store the extracted data
        reviews.append({
            "item": item,
            "company": company,
            "sentiment": sentiment,
            "response_message": response_message
        })

    # Save the extracted data to a CSV file using UTF-8 encoding
    output_file_path = r'C:\Users\sanju\bridgelab_local files\GENAi\review_output.csv'
    with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Item', 'Company', 'Sentiment', 'Response Message']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for review in reviews:
            writer.writerow({
                'Item': review['item'],
                'Company': review['company'],
                'Sentiment': review['sentiment'],
                'Response Message': review['response_message']
            })

    print(f"Reviews have been processed and saved to {output_file_path}")


In [75]:
try:
    if __name__ == '__main__':
        main()
except Exception as e:
    print(e)

429 Resource has been exhausted (e.g. check quota).
